In [1]:
library(Seurat)
seurat = readRDS('prehashing_seurat.RDS')

Warning message:
"package 'Seurat' was built under R version 4.1.3"
Attaching SeuratObject

Attaching sp



In [2]:
seurat = UpdateSeuratObject(seurat)

Updating from v2.X to v3.X

Validating object structure

Updating object slots

Ensuring keys are in the proper strucutre

Ensuring feature names don't have underscores or pipes

Object representation is consistent with the most current Seurat version



In [3]:
results = read.table('citeseq_results_dense.csv', header = 1, stringsAsFactors = FALSE, sep = ',', row.names = 1)

head(results)


,HTO1.ACCCACCAGTAAGAC,HTO2.GGTCGAGAGCATTCA,HTO3.CTTGCCGCATGTCAT,HTO4.AAAGCATTCTTCACG,HTO5.CTTTGTCTTTGTGAG,HTO6.TATGCTGCCACGGTA,HTO7.GAGTCTGCCAGTATC,HTO8.TATAGAACGCCAGGC,HTO9.TGCCTATGAAACAAG,HTO10.CCGATTGTAACAGAC,unmapped
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
O2GTCGAAAGCCTT,0,0,0,0,0,22,32,15,550,212,107
O2ATTGTACGCGAT,0,0,0,0,0,122,29,29,16,16,40
O2CAGGGAAGTCGC,0,0,0,0,0,11,10,5,374,1,82
O2ATTTCGAAACTT,0,0,0,0,0,30,386,304,16,4,110
O2GCTACAGGTACC,0,3,0,0,0,401,13,11,18,7,118
O2GCCGTAGTGAGA,0,0,0,0,0,21,15,6,1,212,55


In [4]:
hash_mat = results
seurat$id = paste0(seurat$orig,  seurat$bc)
head(seurat$id)
seurat = subset(seurat, cells = colnames(seurat)[seurat$id %in% rownames(hash_mat)])

S4_0             S4_1             S4_2             S4_3 
"S4AAAAAACCGTTT" "S4AAAAAATCGATA" "S4AAAAAGCGCTGA" "S4AAAACATCCAAA" 
            S4_4             S4_5 
"S4AAAACATCGTTA" "S4AAAACGCTTTAA"

In [5]:
seurat

An object of class Seurat 
14458 features across 19535 samples within 1 assay 
Active assay: RNA (14458 features, 0 variable features)

In [6]:
hash_mat = results[seurat$id,]
bcs_to_names = colnames(seurat)
names(bcs_to_names) = seurat$id
rownames(hash_mat) = bcs_to_names[rownames(hash_mat)]

In [7]:
seurat[['HTO']] = CreateAssayObject(counts = t(hash_mat))
seurat$letter = substring(seurat$orig, 1, 1)
#seurat = subset(seurat, cells =colnames(seurat)[seurat$n_genes > 1000])

In [8]:
unique(seurat$orig)

[1] "S4" "S5" "S6" "S7"

In [9]:
table(seurat$orig)


  S4   S5   S6   S7 
5447 7190 3204 3694 

In [10]:
library(MASS)
library(cluster)
library(outliers)
library(dplyr)
seurat$hash = NA
Idents(seurat) = 'orig'
for (curr in unique(seurat$orig)) {
    message(curr)
   sub =subset(seurat, idents =  curr)
    
    if (length(colnames(sub)) > 50) {
 
        
        if(curr %in% c('S5', 'S4')) {
            sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(6,7,8,9,10)]))
        }
    
        if(curr %in% c('S7', 'S6')) {
            sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1,2,3,4,5)]))
        }
        
        
        
        sub = NormalizeData(sub, assay = 'HTO', normalization.method = 'CLR', margin = 2)
        
        

        DefaultAssay(sub) = 'HTO'
        data = GetAssayData(sub)

        # k medioids
        meds = pam(t(data), k = dim(data)[1])
        calls = t(data)
        cluster_data = meds$medoids %>% as.matrix()
        

        for (i in 1:dim(cluster_data)[2]) {
            
  

            curr_med = which.max(cluster_data[,i])
            negs = meds$clustering[meds$clustering != curr_med]
            neg_vals = data[i, names(negs)]
            pos_vals = data[i, meds$clustering == curr_med]
            pos_vals = pos_vals[order(pos_vals)] 
            done = FALSE
            thresh = min(pos_vals)
            # remove outliers and set threshold
            while(!done & length(pos_vals) > 5) {
                done = TRUE
                avg = mean(pos_vals)
                diff1 = avg - min(pos_vals)
                diff2 = max(pos_vals) - avg
                test = grubbs.test(pos_vals, opposite = diff1 < diff2)
                if (test$p.value < 0.05) {
                    pos_vals = pos_vals[-1]
                    done = FALSE
                }
                thresh = min(pos_vals)

            }
            print(thresh)
            if (thresh < .3) {
                thresh = 1.5
                message('danger')
            }   

            if (thresh < .3) {
                thresh = 1.5
                message('danger')
            }   
            
            if (curr == 'S7' & i == 2) {
                thresh = 1
            }
            
            if (curr == 'S7' & i == 1) {
                thresh = 2
            }
            
            if (curr == 'S7' & i == 4) {
                thresh = 2
            }
             if (curr == 'S7' & i == 3) {
                thresh = 2
            }
            if (curr == 'S7' & i == 5) {
                thresh = 1.5
            }
            
            
            calls[,i] = as.numeric(calls[,i] >= thresh)
            }
        
        
        assign = apply(calls, 1, which.max)
        assign = paste0( 'HTO', assign)
        rsums = rowSums(calls)
            
        assign[rsums > 1] = 'Doublet'
        assign[rsums == 0] = 'Negative'
 
        seurat@meta.data[colnames(sub), 'hash'] = assign

        pheatmap::pheatmap(t(data), annotation_row = data.frame(row.names = colnames(data), id = factor(assign)), filename = paste0(curr, '.png'), show_rownames = FALSE)

        
    }
}

Warning message:
"package 'outliers' was built under R version 4.1.1"
Warning message:
"package 'dplyr' was built under R version 4.1.3"

Attaching package: 'dplyr'


The following object is masked from 'package:MASS':

    select


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


S4

Normalizing across cells



[1] 1.809497
[1] 2.057133
[1] 2.122157
[1] 0


danger



[1] 1.886501


S5

Normalizing across cells



[1] 1.561516
[1] 1.465398
[1] 1.852091
[1] 1.627846
[1] 1.266535


S6

Normalizing across cells



[1] 1.538262
[1] 1.329744
[1] 1.627762
[1] 1.98161
[1] 1.895404


S7

Normalizing across cells



[1] 1.058688
[1] 0.6170331
[1] 1.213371
[1] 1.638048
[1] 1.070042


In [11]:
saveRDS(seurat, 'hashed_seurat.RDS')

In [12]:
saveRDS(seurat@meta.data, 'hashed_metadata.rds')